# MOFs LM Probing for Linker

In [ ]:
from baseline import PromptSet

from tqdm.auto import tqdm
from nltk.corpus import stopwords

import pandas as pd

from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline, logging

logging.set_verbosity_error()  # avoid irritating transformers warnings

## Read in the Triples

In [ ]:
file = "../data/mof_linkers.csv"

In [ ]:
df = pd.read_csv(file)
df.head()

In [ ]:
df.shape

## Probing BERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
model = AutoModelForMaskedLM.from_pretrained("bert-large-cased")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Create Prompts

In [ ]:
def create_linker_prompt1(sub, mask):
    return f"{sub} contains a {mask} which is organic linker."
    
def create_linker_prompt2(sub, mask):
    return f"{sub} contains an organic linker which is {mask}."

def create_linker_prompt3(sub, mask):
    return f"The metal clusters in {sub} are connected by {mask}."

def create_linker_prompt4(sub, mask):
    return f"{sub} is an MOF made of metal centers connectd by organic linker {mask}."
    
def create_linker_prompt5(sub, mask):
    return f"{sub} is an MOF material containing metal centers and organic linkers {mask}."
    
def create_linker_prompt6(sub, mask):
    return f"{sub} is an MOF structure with metal centers and organic linkers {mask}." 
    
def create_linker_prompt7(sub, mask):
    return f"The organic linker in MOF {sub} is {mask}."
    
def create_linker_prompt8(sub, mask):
    return f"{sub} has an organic linker with SMILES string {mask}."
    
def create_linker_prompt9(sub, mask):
    return f"{sub} has SBUs and organic linkers {mask}."
    
def create_linker_prompt10(sub, mask):
    return f"{sub} is a type of polymer with metal clusters and organic linkers {mask}."
    
def create_linker_prompt11(sub, mask):
    return f"{mask} is the structure of the organic linker in MOF {sub}."
    
def create_linker_prompt12(sub, mask):
    return f"The structure of the organic linker of MOF {sub} is {mask}."

In [ ]:
processors = [create_linker_prompt1, create_linker_prompt2,
              create_linker_prompt3, create_linker_prompt4,
              create_linker_prompt5, create_linker_prompt6,
              create_linker_prompt7, create_linker_prompt8,
              create_linker_prompt9, create_linker_prompt10,
              create_linker_prompt11, create_linker_prompt12
             ]

## Test BERT

In [ ]:
def prompt_probe(model, tokenizer, prompt_processor, df_sub, mask_token, \
                 subjectCol, relationCol, objectCol, top_k=100):

    pipe = pipeline(
        task="fill-mask",
        model=model,
        tokenizer=tokenizer,
        top_k=top_k,
    )

    prompts = PromptSet([prompt_processor(row.SubjectEntity, mask_token) for _, row in df_sub.iterrows()])

    outputs = []
    for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
        outputs.append(out)

    results = []
    for idx, row in df_sub.iterrows():
        for output in outputs[idx]:
            record = {}
            record['SubjectEntity'] = row[subjectCol]
            record['Relation'] = row[relationCol]
            record['ObjectEntity'] = row[objectCol]
            record['prompt']= prompts[idx]
            record['predictedScore'] = output['score']
            record['predictedToken'] = output['token_str']
            results.append(record)

    return pd.DataFrame(results)

In [ ]:
# Compute a column indicating whether the predictedToken in the ObjectEntity list
def hasPredicted(row):
    token = row['predictedToken']
    objectEntities = row['ObjectEntity']
    #print(objectEntities)
    if pd.notna(objectEntities):
        if token.lower() == objectEntities.lower():
            return 1
       
    return 0

In [ ]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [ ]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [ ]:
# Assume the predicted_df has a binary column 'predicted' indicating whether ObjectEntity has been predicted
# 1 for yes, 0 for no
def evaluate_acc_n(predicted_df, relation, prompt_template, at_k=5):
    topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=at_k)
    topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')
    topk_agg_df.columns = ['SubjectEntity', 'predicted']
    result = {}
    result['Relation'] = relation
    result['prompt_template'] = prompt_template
    result['acc'] = topk_agg_df.predicted.mean()
    result['at_k'] = at_k

    return result

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasLinker', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasLinker_BERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasLinker_BERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing MatBERT

In [ ]:
# MatBERT
tokenizer = AutoTokenizer.from_pretrained("../model/model/matbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("../model/model/matbert-base-cased")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasLinker', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasLinker_MatBERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasLinker_MatBERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing MatSciBERT

In [ ]:
# MatSciBERT
tokenizer = AutoTokenizer.from_pretrained('m3rg-iitd/matscibert')
model = AutoModelForMaskedLM.from_pretrained('m3rg-iitd/matscibert')

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasLinker', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasLinker_MatSciBERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasLinker_MatSciBERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing SciBERT

In [ ]:
# SciBERT
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModelForMaskedLM.from_pretrained('allenai/scibert_scivocab_cased')

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasLinker', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasLinker_SciBERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasLinker_SciBERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing RoBERTa

In [ ]:
# RoBERTa
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
model = AutoModelForMaskedLM.from_pretrained("roberta-large")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasLinker', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasLinker_RoBERTa.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasLinker_RoBERTa_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)